In [5]:
import sys
sys.path.append("../src")
from data_loader import load_track_json_files, extract_mood_labeled_tracks, map_tags_to_mood

# Set path to your subset
folder = "../data/raw/lastfm_subset"

# Load all JSON track files
all_tracks = load_track_json_files(folder)

# Filter to only mood-related tracks
mood_tracks = extract_mood_labeled_tracks(all_tracks)

# Assign mood label
for t in mood_tracks:
    t["mood"] = map_tags_to_mood(t["tags"])

import pandas as pd
df = pd.DataFrame(mood_tracks)
print(f"Found {len(df)} mood-tagged tracks.")
df


Found 2111 mood-tagged tracks.


,track_id,title,artist,tags,mood
0,TRAAAEF128F4273421,Something Girls,Adam Ant,[strangeromanticdeadrockcave],emotional
1,TRAABLR128F423B7E3,Floating,Blue Rodeo,[blues],sad
2,TRAABNV128F425CEE1,High Tide,Richard Souther,"[drjazzmrfunkmusic, peaceful]",happy
3,TRAABVM128F92CA9DC,Caught In A Dream,Tesla,"[chillout, chillout rock, blues rock, Power ba...",calm
4,TRAACER128F4290F96,Setting Fire to Sleeping Giants,The Dillinger Escape Plan,"[melodic, more fun than a stick in the eye, me...",calm
...,...,...,...,...,...
2106,TRBIJFB128F92ED124,On the Road (to Damnation),It Dies Today,[melodic metalcore],calm
2107,TRBIJIA128F425F57D,The Forgotten Lands,David Arkenstone,"[chillout, relaxing, chill]",calm
2108,TRBIJLT12903CE7070,God Don't Never Change,Blind Willie Johnson,"[blues, Classic Blues, Old Blues, gospel blues...",sad
2109,TRBIJMU12903CF892B,The Hanged Man,Moonspell,"[sad, romantic, melodic, sadness]",sad
